首先还是读取我们的数据集，这里我们对数据做一个归一化的处理，方便模型收敛，并且对label做一个one-hot处理，因为是10分类问题，所以num_classes这个参数的值为10。

In [18]:
from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

模型搭建阶段我们依旧采用序贯模型Sequential，整体的结构上文已经提过，这里不再赘述。

In [19]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

然后是定义损失函数、优化方法、评估指标。

In [20]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 8, 8, 64)          0         
__________

模型准备完成后就可以开始训练了，这里我们引入了EarlyStopping，EarlyStopping可以保证模型在准确率不在提升的前提下提前结束训练，其中monitor参数表示的是停止时参考的指标，这里我们采用的是准确率，patience的意思是多少轮指标没有改变模型训练就停止，这里我们使用的是5，也就是说如果训练五轮，准确率依旧没有提升即停止训练。

In [21]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=5, verbose=1)
model.fit(x_train, y_train, batch_size=64, epochs=200, callbacks=[early_stopping])

Epoch 1/200
50000/50000 [==============================] - 20s 407us/step - loss: 1.7263 - acc: 0.3455
Epoch 2/200
50000/50000 [==============================] - 19s 385us/step - loss: 1.3489 - acc: 0.5065
Epoch 3/200
50000/50000 [==============================] - 19s 384us/step - loss: 1.1491 - acc: 0.5922
Epoch 4/200
50000/50000 [==============================] - 19s 384us/step - loss: 1.0275 - acc: 0.6387
Epoch 5/200
50000/50000 [==============================] - 19s 383us/step - loss: 0.9471 - acc: 0.6709
Epoch 6/200
50000/50000 [==============================] - 19s 384us/step - loss: 0.8785 - acc: 0.6952
Epoch 7/200
50000/50000 [==============================] - 19s 383us/step - loss: 0.8301 - acc: 0.70911s - los
Epoch 8/200
50000/50000 [==============================] - 19s 383us/step - loss: 0.8024 - acc: 0.72151s - loss: 0
Epoch 9/200
50000/50000 [==============================] - 19s 383us/step - loss: 0.7752 - acc: 0.7321
Epoch 10/200
50000/50000 [===========================

最后我们就可以在测试集进行测试了，可以看到最后的准确率在79.28%，当然了，读者也可以自行对上文的VGG参数进行调整来提高模型的最终效果。

In [22]:
score = model.evaluate(x_test, y_test, batch_size=64)
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 138us/step
Test accuracy: 0.7918
